In [1]:
import pandas as pd

data=pd.read_csv('CP1_df_data.csv')
data.drop(['Unnamed: 0'], axis=1, inplace=True)
data

,passage,summary
0,"새해 주요 기업들의 신년사 화두는 단연 ‘생존’이었다., 미래 먹거리 부재, 중국...",새해 주요 기업들의 신년사 화두는 단연 생존이었고 미래 먹거리 부재 규제로 인한 신...
1,매년 빠지지 않는 새해 3대 결심으로 금주·금연·다이어트가 꼽힐 정도로 건강 문제는...,엔도르핀 등을 분비하고 면역력을 강화하는 효과가 있는 웃음에 대해 자생한방병원 박 ...
2,지난해 12월 31일 성동구 마장동의 한 아파트 지하 공용 보일러실에서 불이 나 주...,성동구 마장동의 한 아파트 지하 공용 보일러실에서 불이 나 가스와 전기 수도가 끊겼...
3,"“반도체 가격이 상승 전환될 것이라고 보는 게 글로벌 전망 기관의 지배적 견해다.,...",정부는 반도체 업황 회복 가능성에 크게 기대하고 있으며 여러 전문가과 기관도 반도체...
4,학대·유기·이혼·빈곤 등의 이유로 아동보호시설에 보내진 영·유아가 열악한 환경에서 ...,육아정책연구소 이 연구위원은 아동보호시설에 대한 보고서에서 영유아가 열악한 환경에서...
...,...,...
1995,"훌루말레의 수상 비행장.,이곳은 장거리를 여행하는 사람들이 이용하는 곳이다.,특히 ...",호화 리조트로 유명한 낭갈리 섬에는 수상레저를 즐기는 사람들이 많다.
1996,"안내인은 금 채취의 특징을 설명했다.,이곳의 금 채취는 슬루싱이라는 방식을 사용했단...",뉴질랜드 서부는 골드러시가 일어나면서 개발되기 시작했고 1876년을 마지막으로 금광...
1997,"흐름을 거스르는 일은 쉽지 않아 보인다.,나일강 물결 위에서 할 수 있는 스포츠가 ...",나일강은 레프팅 마니아들에게 세계적인 명소이다.
1998,"잠시 마음을 놓는 순간 거세지는 게 ,이곳의 바람.,마을을 떠나는 길에는 구름까지 ...",마젤란 펭귄은 매년 같은 집에서 같은 암수가 만나 새끼를 낳고 여름 내내 수영을 가...


In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.7 MB/s eta 0:00:00


In [2]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BartForConditionalGeneration, BartTokenizerFast



# BART 모델 불러오기
model = BartForConditionalGeneration.from_pretrained("hyunwoongko/kobart")

# Tokenizer 정의하기
tokenizer = BartTokenizerFast.from_pretrained("hyunwoongko/kobart")

# 학습용 DataLoader 정의하기
class Dataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        row = self.data.iloc[index]
        text = row['passage']
        summary = row['summary']
        input_ids = self.tokenizer.encode(text, max_length=self.max_length, truncation=True, padding='max_length', return_tensors='pt')
        output_ids = self.tokenizer.encode(summary, max_length=self.max_length, truncation=True, padding='max_length', return_tensors='pt')
        return {'input_ids': input_ids.squeeze(),  'decoder_input_ids': output_ids.squeeze()[:-1], 'decoder_output_ids': output_ids.squeeze()[1:]}



batch_size = 4
dataset = Dataset(data, tokenizer, max_length=500)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)





You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'PreTrainedTokenizerFast'. 
The class this function is called from is 'BartTokenizerFast'.


In [3]:
# 학습하기
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.train()

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
num_epochs = 1
for epoch in range(num_epochs):
  for batch in dataloader:
          input_ids = batch['input_ids'].to(device)
       
          decoder_input_ids = batch['decoder_input_ids'].to(device)
        
          decoder_output_ids = batch['decoder_output_ids'].to(device)
        
          outputs = model(input_ids=input_ids, decoder_input_ids=decoder_input_ids, labels=decoder_output_ids)
          loss = outputs.loss
    
          loss.backward()
          optimizer.step()
          optimizer.zero_grad()
# for epoch in range(num_epochs):
#    for batch in dataloader:
#          batch_input_ids = batch['input_ids'].to(device)
#          #batch_decoder_input_ids = batch['decoder_input_ids'].to(device)
#          batch_labels = batch['decoder_output_ids'].to(device)
#          #output=model(batch_labels)
#          optimizer.step()
#          optimizer.zero_grad()
        
    

# 모델 저장하기
model_path = 'summarization2.pt'
model.save_pretrained(model_path)

In [11]:
import torch
from transformers import BartForConditionalGeneration, BartTokenizerFast

model_path = 'summarization2.pt'
model = BartForConditionalGeneration.from_pretrained(model_path)
tokenizer = BartTokenizerFast.from_pretrained("hyunwoongko/kobart")

def generate_summary(input_text):
    input_ids = tokenizer.encode(input_text, max_length=500, truncation=True, padding='max_length', return_tensors='pt').to(device)
    summary_ids = model.generate(input_ids, max_length=128, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary
data = "본 연구는 코로나19 이후 초등학생의 신체활동량과 신체적자기개념, 그리고 긍정심리자본의 관계를 탐색하고자 하였다. 구체적으로 운동참여가 긍정심리자본에 영향력을 행사하는 과정에서 신체적자기개념의 매개효과를 검증하고자 하였다. 이를 위해 초등학교 6학년에 재학 중인 97명의 학생들을 대상으로 주간운동빈도, 주간운동시간, 신체적자기개념, 긍정심리자본을 측정하는 설문조사를 실시하였고, 수집된 자료를 바탕으로 기술통계분석, 상관분석, 구조방정식 모형 검증을 실시하였다. 본 연구를 수행한 결과, 초등학생들의 운동참여수준(주간운동빈도, 주간운동시간) 은 신체적자기개념 및 긍정심리자본의 모든 하위요인들과 정적 상관관계가 나타났다. 또한 신체적자기개념은 운동참여수준과 긍정심리자본의 관계를 부분적으로 매개하는 것으로 나타났다. 본 연구의 결과를 통해 운동에 자주 참여하거나 운동에 참여하는 시간이 많은 학생일수록 자신의 신체에 대해서 긍정적으로 인식하고, 일상생활에서의 희망, 낙관성, 탄력성, 효능감을 가진다는 것을 확인하였으며, 초등학생들이 운동에 참여함으로써 향상된 신체적자기개념이 긍정심리자본의 향상으로 이어질 수 있다는 것을 실증적으로 규명하였다."
generate_summary(data)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'PreTrainedTokenizerFast'. 
The class this function is called from is 'BartTokenizerFast'.


'본 연구는 코로나19 이후 초등학생의 운동참여가 긍정심리자본에 영향력을 행사하는 과정에서 신체적자기개념의 매개효과를 검증하고자 하였다.'

In [ ]:
import torch
from transformers import BartForConditionalGeneration, BartTokenizerFast

class SummarizationModel:
    def __init__(self, model_path):
        model = BartForConditionalGeneration.from_pretrained(model_path)
        tokenizer = BartTokenizerFast.from_pretrained("hyunwoongko/kobart")

    def generate_summary(self, text):
        input_ids = tokenizer.encode(text, max_length=500, truncation=True, padding='max_length', return_tensors='pt').to(device)
        summary_ids = model.generate(input_ids, max_length=128, num_beams=4, early_stopping=True)
        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        return summary


In [6]:
import torch
from transformers import BartForConditionalGeneration

class SummarizationModel:
    def __init__(self, model_path):
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model = BartForConditionalGeneration.from_pretrained(model_path)
        self.model.to(self.device)
        self.model.eval()
        
    def summarize(self, text):
        input_ids = self.tokenizer.encode(text, max_length=500, truncation=True, padding='max_length', return_tensors='pt').to(self.device)
        summary_ids = self.model.generate(input_ids=input_ids, max_length=150, num_beams=2, early_stopping=True)
        summary = self.tokenizer.decode(summary_ids.squeeze(), skip_special_tokens=True)
        return summary


In [21]:
import torch
from transformers import BartForConditionalGeneration

model_path = 'summarization2.pt'
model = BartForConditionalGeneration.from_pretrained(model_path)

torch.save(model.state_dict(), 'summarization2.py')


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [22]:
!mkdir summarization_package


In [23]:
!mv summarization2.pth summarization_package/


In [24]:
!touch summarization_package/__init__.py


In [33]:
!touch summarization_package/setup.py

touch: cannot touch 'summarization_package/setup.py': No such file or directory


In [25]:
%pwd

'/content'

In [34]:
%cd summarization_package/setup.py

[Errno 2] No such file or directory: 'summarization_package/setup.py'
/content/summarization_package


In [35]:
!touch summarization_package/setup.py


touch: cannot touch 'summarization_package/setup.py': No such file or directory


In [36]:
# setup.py
from setuptools import setup, find_packages

setup(
    name='summarization_package',
    version='0.1',
    packages=find_packages(),
    install_requires=[
        'torch',
        'transformers'
    ]
)


SystemExit: ignored

/usr/local/lib/python3.9/dist-packages/IPython/core/interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [39]:
!python setup.py sdist bdist_wheel


running sdist
running egg_info
creating summarization2.egg-info
writing summarization2.egg-info/PKG-INFO
writing dependency_links to summarization2.egg-info/dependency_links.txt
writing requirements to summarization2.egg-info/requires.txt
writing top-level names to summarization2.egg-info/top_level.txt
writing manifest file 'summarization2.egg-info/SOURCES.txt'
error: package directory 'summarization2' does not exist
